# Neural Network

In [1]:
# Isto importa numpy, que é uma biblioteca de álgebra linear.
# Esta é a nossa única dependência.
import numpy as np

In [2]:
# Esta é a nossa "não linearidade".
# Embora possa haver vários tipos de funções, essa não linearidade mapeia uma função chamada "sigmóide".
# Uma função sigmóide mapeia qualquer valor para um valor entre 0 e 1.
# Nós a usamos para converter números em probabilidades.
# Ele também possui várias outras propriedades desejáveis para o treinamento de redes neurais.
def nonlin(x, deriv=False):
    # Observe que essa função também pode gerar a derivada de um sigmóide (quando deriv = True).
    # Uma das propriedades desejáveis de uma função sigmóide é que sua saída pode ser usada para criar sua derivada.
    # Se a saída do sigmoide for uma variável "out", a derivada será simplesmente out * (1-out). Isto é muito eficiente.
    # Se você não é familiar com derivadas, pense nisso como a inclinação da função sigmóide em um determinado ponto.
    if deriv == True:
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))

In [3]:
# Isso inicializa nosso conjunto de dados de entrada como uma matriz numpy.
# Cada linha é um único "exemplo de treinamento".
# Cada coluna corresponde a um dos nossos nós de entrada.
# Assim, temos 3 nós de entrada na rede e 4 exemplos de treinamento.
X = np.array([[0,0,1],
              [0,1,1],
              [1,0,1],
              [1,1,1]])

In [4]:
# Isso inicializa nosso conjunto de dados de saída.
# Nesse caso, gerei o conjunto de dados horizontalmente (com uma única linha e 4 colunas) para espaço.
# ".T" é a função de transposição.
# Após a transposição, essa matriz y possui 4 linhas com uma coluna.
# Assim como nossa entrada, cada linha é um exemplo de treinamento e cada coluna (apenas uma) é um nó de saída.
# Portanto, nossa rede possui 3 entradas e 1 saída.        
Y = np.array([[0,0,1,1]]).T

In [5]:
# É uma boa prática propagar seus números aleatórios.
# Seus números ainda serão distribuídos aleatoriamente,
# mas serão distribuídos aleatoriamente exatamente da mesma maneira sempre que você treinar.
# Isso facilita ver como suas alterações afetam a rede.
np.random.seed(1)

In [6]:
# Esta é a nossa matriz de peso para esta rede neural.
# É chamado "syn0" para implicar "sinapse zero".
# Como temos apenas 2 camadas (entrada e saída), precisamos apenas de uma matriz de pesos para conectá-las.
# Sua dimensão é (3,1) porque temos 3 entradas e 1 saída.
# Outra maneira de ver é que l0 é do tamanho 3 e l1 é do tamanho 1.
# Assim, queremos conectar todos os nós em l0 a todos os nós em l1,
# o que requer uma matriz de dimensionalidade (3,1).

# Observe também que ele é inicializado aleatoriamente com uma média de zero.
# Há um pouco de teoria que entra na inicialização do peso.
# Por enquanto, considere como prática recomendada que seja uma boa idéia ter uma média de zero na inicialização do peso.

# Outra observação é que a "rede neural" é realmente apenas essa matriz.
# Temos "camadas" l0 e l1, mas são valores transitórios com base no conjunto de dados.
# Nós não os salvamos. Todo o aprendizado é armazenado na matriz syn0.
syn0 = 2 * np.random.random((3,1)) - 1

In [7]:
# Isso inicia nosso código de treinamento de rede real.
# O loop for "itera" várias vezes no código de treinamento para otimizar nossa rede para o conjunto de dados.
for iter in range(10000):
    # Desde a nossa primeira camada "X", são simplesmente nossos dados.
    # Descrevemos explicitamente como tal neste momento.
    # Lembre-se de que X contém 4 exemplos de treinamento (linhas).
    # Vamos processar todos eles ao mesmo tempo nesta implementação.
    # Isso é conhecido como treinamento em "lote inteiro".
    # Portanto, temos 4 linhas diferentes de X, mas você pode pensar nisso como um único exemplo de treinamento, se desejar.
    # Não faz diferença neste momento. (Poderíamos carregar 1000 ou 10.000, se quiséssemos, sem alterar nenhum código).
    l0 = X
    # Este é o nosso passo de previsão.
    # Basicamente, primeiro deixamos a rede "tentar" prever a saída dada a entrada.
    # Em seguida, estudaremos o desempenho, para que possamos ajustá-lo para melhorar um pouco a cada iteração.
    # Esta linha contém 2 etapas. A primeira matriz multiplica 10 por syn0.
    # O segundo passa nossa saída pela função sigmóide.
    # Considere as dimensões de cada um:
    # (4 x 3) ponto (3 x 1) = (4 x 1)
    # A multiplicação da matriz é ordenada, de modo que as dimensões no meio da equação devem ser as mesmas.
    # A matriz final gerada é, portanto, o número de linhas da primeira matriz e o número de colunas da segunda matriz.
    # Como carregamos 4 exemplos de treinamento, terminamos com 4 palpites para a resposta correta, uma matriz (4 x 1).
    # Cada saída corresponde ao palpite da rede para uma determinada entrada.
    # Talvez se torne intuitivo o motivo pelo qual poderíamos "carregar" um número arbitrário de exemplos de treinamento.
    # A multiplicação da matriz ainda funcionaria.
    l1 = nonlin(np.dot(l0,syn0))
    # Portanto, dado que l1 tinha um "palpite" para cada entrada.
    # Agora podemos comparar quão bem ele foi subtraindo a resposta verdadeira (Y) da suposição (l1).
    # l1_error é apenas um vetor de números positivos e negativos que refletem o quanto a rede perdeu.
    l1_error = Y - l1
    # Agora estamos chegando às coisas boas!
    # Este é o molho secreto!
    # Há muita coisa acontecendo nessa linha, então vamos dividir ainda mais em duas partes.
    
    # nonlin(l1,True)
    # Se l1 representa esses três pontos, o código acima gera as inclinações das linhas abaixo.
    # Observe que valores muito altos, como x = 2,0 (ponto verde), e valores muito baixos, como x = -1,0 (ponto roxo) têm inclinações bastante rasas.
    # A inclinação mais alta que você pode ter é x = 0 (ponto azul). Isso desempenha um papel importante.
    # Observe também que todos os derivativos estão entre 0 e 1.
    
    # Existem maneiras mais "matematicamente precisas" do que "a derivada ponderada pelo erro", mas acho que isso captura a intuição.
    # l1_error é uma matriz (4,1). nonlin (l1, True) retorna uma matriz (4,1). O que estamos fazendo é multiplicá-los "elementwise".
    # Isso retorna uma matriz (4,1) l1_delta com os valores multiplicados.
    # Quando multiplicamos as "inclinações" pelo erro, estamos reduzindo o erro de previsões de alta confiança.
    # Veja a imagem sigmóide novamente! Se a inclinação era realmente rasa (perto de 0), a rede tinha um valor muito alto ou muito baixo.
    # Isso significa que a rede estava bastante confiante de uma maneira ou de outra.
    # No entanto, se a rede adivinhou algo próximo a (x = 0, y = 0,5), não ficou muito confiante.
    # Atualizamos essas previsões "insolentes" com mais intensidade e tendemos a deixar as confiantes em paz, multiplicando-as por um número próximo a 0.
    l1_delta = l1_error * nonlin(l1,True)
    # Agora estamos prontos para atualizar nossa rede! Vamos dar uma olhada em um único exemplo de treinamento.
    
    syn0 += np.dot(l0.T,l1_delta)

In [10]:
print("Output After Training:")
print(l1)

Output After Training:
[[0.00966449]
 [0.00786506]
 [0.99358898]
 [0.99211957]]


In [9]:
syn0

array([[ 9.67299303],
       [-0.2078435 ],
       [-4.62963669]])